In [1]:
import pandas as pd
import numpy as np

In [2]:
시총1 = pd.read_csv('./datasets/kospi_시총.csv', encoding='cp949')
시총2 = pd.read_csv('./datasets/kospi_시총2.csv')
시총3 = pd.read_csv('./datasets/df3.csv')

In [3]:
시총 = pd.concat([시총1, 시총2, 시총3]).reset_index(drop=True)
시총['날짜'] = pd.to_datetime(시총['날짜'])
시총['month'] = 시총['날짜'].dt.month
시총 = 시총[시총['month'] == 6]
시총['연도_월'] = 시총['날짜'].dt.strftime('%Y-%m')
시총.columns = ['날짜','시가총액','거래량', '거래대금', '상장주식수', '거래소코드', 'month', '연도_월']
시총['시가총액'] = 시총['시가총액'] / 1000

In [4]:
재무정보 = pd.read_csv('./datasets/재무정보.csv', encoding='cp949')
회사명 = list(재무정보['회사명'].unique())
재무정보['회계년도'] = pd.to_datetime(재무정보['회계년도'])
재무정보['연도_월'] = 재무정보['회계년도'].dt.strftime('%Y-%m')

C:\Users\USER\AppData\Local\Temp\ipykernel_20908\2152901359.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  재무정보['회계년도'] = pd.to_datetime(재무정보['회계년도'])


In [5]:
merged_df = pd.merge(시총[['시가총액', '거래소코드', '연도_월']], 재무정보, on=['거래소코드', '연도_월'])

In [6]:
merged_df.columns = ['시가총액', '거래소코드', '연도_월', '회사명', '회계년도', '유동자산', '부채',
       'PER(최고)', 'PER(최저)', 'PBR(최고)', 'PBR(최저)',
       'PCR(최고)', 'PCR(최저)', 'PSR(최고)', 'PSR(최저)',
       '1주당순이익', '당기순이익']

In [10]:
def make_indicator(회사명):
    df_list = []
    for i in 회사명:
        df = merged_df[merged_df['회사명'] == i].reset_index(drop=True)
#         df = df.dropna(subset=['1주당순이익'])
        df['1Y_EPS'] = ((df['1주당순이익'] - df['1주당순이익'].shift(1)) / df['1주당순이익'].shift(1))
        df['3Y_EPS_mean'] = df['1Y_EPS'].rolling(3).mean()
        df['PER_mean'] = (df['PER(최고)'] + df['PER(최저)']) / 2
        df['PEG'] = df['PER_mean'] / df['3Y_EPS_mean']
        df['NCAV'] = (df['유동자산'] - df['부채'])/df['시가총액']
#         print(df)
        df_list.append(df)
    return df_list

In [11]:
df = make_indicator(회사명)
df = pd.concat(df)
df = df.dropna(subset=['PEG', 'NCAV']).reset_index(drop=True)

In [12]:
df['year'] = df['회계년도'].dt.year

In [13]:
# 날짜별로 묶기
year_list = np.arange(2000, 2024)
year_list

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

In [14]:
df = df[(df['PEG'] > 0)]

In [15]:
results = []
for i in year_list:
    year_df = df[df['year'] == i].reset_index(drop=True)
    if len(year_df) >0 :
        result = {'가치주' : year_df.sort_values('NCAV', ascending=False)[['회사명', 'NCAV']].head(10), '성장주' : year_df.sort_values('PEG', ascending=True)[['회사명', 'PEG']].head(10)}
        results.append(result)        
        
#     print(year_df.sort_values('NCAV', ascending=False)[['회사명', 'NCAV']].head(10))
#     print(year_df.sort_values('PEG', ascending=True)[['회사명', 'PEG']].head(10))


In [16]:
results

[{'가치주':            회사명      NCAV
  207    삼성공조(주)  2.117029
  140    부국철강(주)  1.412785
  111    남양유업(주)  1.157697
  220  미창석유공업(주)  1.117298
  109    조광피혁(주)  1.048597
  206      (주)우성  1.018957
  215     (주)위스컴  1.011836
  75     계양전기(주)  0.982950
  143    (주)신도리코  0.971087
  138   (주)세우글로벌  0.970333,
  '성장주':                회사명       PEG
  56         (주)한창제지  0.152494
  90           (주)혜인  0.349613
  83         대한유화(주)  0.587647
  139     엘지디스플레이(주)  0.683374
  185       금호타이어(주)  0.970032
  51        한미반도체(주)  0.979135
  131        유성기업(주)  1.077060
  4    에이치엘디앤아이한라(주)  1.167184
  152   에스케이디스커버리(주)  1.465219
  16      현대엘리베이터(주)  1.705423},
 {'가치주':            회사명      NCAV
  194    삼성공조(주)  2.001832
  8      화성산업(주)  1.791723
  127    부국철강(주)  1.490035
  193      (주)우성  1.414655
  126      (주)조흥  1.317929
  205  미창석유공업(주)  1.158843
  100    경인전자(주)  1.069017
  2     (주)콤텍시스템  1.037937
  72    (주)동방아그로  1.018676
  94     조광피혁(주)  0.954080,
  '성장주':           회사명       PEG
  152  

+ 2010